In [1]:
import hashlib

class KPIOSimulator:
    def __init__(self):
        # The 'Key Cache' inside the SSD
        self.key_cache = {} 
        self.storage = {}

    def inject_key(self, key_tag, key_value):
        """Host injects a key into the drive's volatile cache."""
        self.key_cache[key_tag] = key_value
        print(f"[DRIVE] Key Tag '{key_tag}' cached.")

    def write_io(self, address, data, key_tag):
        """Simulate a Write I/O command with a Key Tag."""
        if key_tag not in self.key_cache:
            print(f"[ERROR] Invalid Key Tag {key_tag}. Write failed.")
            return

        # Encrypt data using the specific key for this I/O
        key = self.key_cache[key_tag]
        encrypted_data = "".join(chr(ord(c) ^ ord(key[i % len(key)])) for i, c in enumerate(data))
        
        self.storage[address] = {"data": encrypted_data, "tag": key_tag}
        print(f"[IO] Written to {address} using Key Tag {key_tag}")

    def read_io(self, address, key_tag):
        """Simulate a Read I/O command. Must provide the matching key."""
        if address not in self.storage:
            return "Empty"
        
        stored_entry = self.storage[address]
        key = self.key_cache.get(key_tag, "")
        
        # Attempt decryption
        raw_data = stored_entry['data']
        decrypted = "".join(chr(ord(c) ^ ord(key[i % len(key)])) for i, c in enumerate(raw_data))
        
        return decrypted

# --- RUNNING THE DEMO ---
drive = KPIOSimulator()

# 1. Host injects keys for two different 'Tenants'
drive.inject_key(key_tag="TENANT_A", key_value="secret_alpha")
drive.inject_key(key_tag="TENANT_B", key_value="secret_beta")

# 2. Tenant A writes a sensitive property valuation
drive.write_io("LBA_001", "Sentinel Building: $50M", key_tag="TENANT_A")

# 3. Tenant B writes a different valuation
drive.write_io("LBA_002", "Clock Tower: $30M", key_tag="TENANT_B")

# 4. Attempt to read Tenant A's data using Tenant B's key (The KPIO protection)
leak_attempt = drive.read_io("LBA_001", key_tag="TENANT_B")
print(f"\nUnauthorized Read Result: {leak_attempt}") # This will be gibberish

# 5. Correct Read
proper_read = drive.read_io("LBA_001", key_tag="TENANT_A")
print(f"Authorized Read Result:   {proper_read}")

[DRIVE] Key Tag 'TENANT_A' cached.
[DRIVE] Key Tag 'TENANT_B' cached.
[IO] Written to LBA_001 using Key Tag TENANT_A
[IO] Written to LBA_002 using Key Tag TENANT_B

Unauthorized Read Result: Sentineo)F|{zbxyv -!V
Authorized Read Result:   Sentinel Building: $50M
